In [1]:
import glymur
from glob import glob
import numpy as np
from xml.etree.ElementTree import QName
import xml.etree.ElementTree
from scipy.ndimage.interpolation import zoom
from time import time, sleep, gmtime
from os import path
import os
import errno
import dill
from os import devnull
import sys
import warnings
import argparse
import multiprocessing
import traceback
import gdal
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool
import builtins
from copy import deepcopy
from psutil import virtual_memory
from tempfile import TemporaryFile
import resource
import matplotlib
from stopit import ThreadingTimeout as Timeout
from stopit import TimeoutException
matplotlib.use("agg")
import matplotlib.pyplot as plt
from skimage.exposure import rescale_intensity, adjust_gamma
from threading import Thread

In [2]:
# !./cB4S2.py -h

In [3]:
pn = '/media/SOLabNFS2/tmp/sentinel-2_test/S2A_OPER_PRD_MSIL1C_PDMC_20160108T151349_R135_V20160108T082023_20160108T082023.SAFE/'

In [4]:
import os
granules = os.listdir(pn + 'GRANULE/')

In [5]:
fn = granules[0]

In [6]:
from cB4S2 import S2_MSI_Image

In [7]:
import numpy as np

In [ ]:
Sentinel-2 Bands	Wavelength (µm)	Resolution (m)
Band 1  - Coastal aerosol		0.443	60

Band 2  - Blue					0.490	10
Band 3  - Green					0.560	10
Band 4  - Red					0.665	10

Band 5  - Vegetation Red Edge	0.705	20
Band 6  - Vegetation Red Edge	0.740	20
Band 7  - Vegetation Red Edge	0.783	20

Band 8  - NIR					0.842	10

Band 8A - Vegetation Red Edge	0.865	20

Band 9  - Water vapour			0.945	60
Band 10 - SWIR - Cirrus			1.375	60

Band 11 - SWIR					1.610	20
Band 12 - SWIR					2.190	20

In [17]:
S2_MSI_granule_path = pn + 'GRANULE/' + fn + '/'
# namespace           = '/home/mag/Documents/repos/solab/PySOL/notebooks/Sentinel-2/S2_PDI_Level-1C_Tile_Metadata.xsd'
namespace="https://psd-12.sentinel2.eo.esa.int/PSD/S2_PDI_Level-1C_Tile_Metadata.xsd"
# import_bands=['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12']
import_bands        = ["B04", "B03", "B02"]
target_resolution=10.0
numpy_type=np.float16
interpolation_order=1
data_mode="dense"
# driver="Jasper"
driver = "OpenJpeg200"

In [18]:
S2_img = S2_MSI_Image(S2_MSI_granule_path = S2_MSI_granule_path,
                      import_bands        = import_bands,
                      data_mode           = "dense",
                      target_resolution   = target_resolution,
                      numpy_type          = numpy_type,
                      interpolation_order = interpolation_order)

Load S2 MSI data from:/media/SOLabNFS2/tmp/sentinel-2_test/S2A_OPER_PRD_MSIL1C_PDMC_20160108T151349_R135_V20160108T082023_20160108T082023.SAFE/GRANULE/S2A_OPER_MSI_L1C_TL_SGS__20160108T115454_A002849_T35HMA_N02.01/
Final shape for each channel: [10980, 10980]
Reading: /media/SOLabNFS2/tmp/sentinel-2_test/S2A_OPER_PRD_MSIL1C_PDMC_20160108T151349_R135_V20160108T082023_20160108T082023.SAFE/GRANULE/S2A_OPER_MSI_L1C_TL_SGS__20160108T115454_A002849_T35HMA_N02.01/IMG_DATA/S2A_OPER_MSI_L1C_TL_SGS__20160108T115454_A002849_T35HMA_B04.jp2
Read band B04 in 31.38s, pure load time:27.58s, resample time: 3.80s, zoom: 1.000, final shape: [10980, 10980], index: 0
Reading: /media/SOLabNFS2/tmp/sentinel-2_test/S2A_OPER_PRD_MSIL1C_PDMC_20160108T151349_R135_V20160108T082023_20160108T082023.SAFE/GRANULE/S2A_OPER_MSI_L1C_TL_SGS__20160108T115454_A002849_T35HMA_N02.01/IMG_DATA/S2A_OPER_MSI_L1C_TL_SGS__20160108T115454_A002849_T35HMA_B03.jp2
Read band B03 in 31.61s, pure load time:27.84s, resample time: 3.77s, z

In [19]:
S2_rgb = S2_img.S2_image_to_rgb(rgb_bands=import_bands)

Rescale band for RGB image: 0,B04,(0.02,0.06)->(0,1)
Rescale band for RGB image: 1,B03,(0.04,0.07)->(0,1)
Rescale band for RGB image: 2,B02,(0.08,0.11)->(0,1)


In [20]:
S2_img.save_rgb_image(S2_rgb, fn+'.png')

In [ ]:
scale = 7
plt.imshow(S2_img.data[:,:,0][::scale,::scale], vmin=0, vmax=0.05)
plt.colorbar()

In [ ]:
rgb_gamma=(1.0, 1.0, 1.0)
pixel_dd_hist=20
pixel_dd=1
hist_chop_off=0.01
band = "B11"

In [ ]:
i_band = S2_img.band_list.index(band)
hh, xx = np.histogram(S2_img.data[::pixel_dd_hist, ::pixel_dd_hist, i_band], bins=300)
hh = np.double(hh)
bb = 0.5 * (xx[1:] + xx[:-1])


In [ ]:
hh = hh / np.max(hh)
lim = (lambda x: (np.min(x), np.max(x)))(bb[hh > hist_chop_off])

In [ ]:
lim

In [ ]:
scale = 7
plt.imshow(S2_rgb[:,:,:][::scale,::scale])
plt.colorbar()

In [ ]:
class L(object):
    @staticmethod
    def _band_name(fn):
        return fn.split(".jp2")[0].split("_")[-1]
self = L()

In [ ]:
def __zeros__(shape, dtype, max_mem_frac=0.3):
    def in_memory_array(shape, dtype):
        return np.zeros(shape, dtype)

In [ ]:
glob(path.join(S2_MSI_granule_path, "S2*.xml"))

In [ ]:
self.driver = driver
self.target_resolution = target_resolution
self.full_band_list = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11',
                       'B12', ]
self.band_list = list(import_bands)

print("Load S2 MSI data from:%s" % S2_MSI_granule_path)

# take XML file in folder above IMG_DATA
xml_file_name = glob(path.join(S2_MSI_granule_path, "S2*.xml"))[0]

bf = xml.etree.ElementTree.parse(xml_file_name).getroot()

# extract geo coding info from XML file
geo_codings = bf.find(str(QName(namespace, 'Geometric_Info'))).find("Tile_Geocoding")

self.metadata = {}
self.metadata["HORIZONTAL_CS_NAME"] = geo_codings.find("HORIZONTAL_CS_NAME").text
self.metadata["HORIZONTAL_CS_CODE"] = geo_codings.find("HORIZONTAL_CS_CODE").text

# take XML file in folder above GRANULE Folder
xml_file_name = glob(path.join(path.dirname(path.dirname(path.dirname(S2_MSI_granule_path))), "S2*.xml"))[0]
bf = xml.etree.ElementTree.parse(xml_file_name).getroot()

band_map = {'B1': 'B01', 'B2': 'B02', 'B3': 'B03', 'B4': 'B04', 'B5': 'B05', 'B6': 'B06', 'B7': 'B07',
            'B8': 'B08',
            'B9': 'B09', 'B10': 'B10', 'B11': 'B11', 'B12': 'B12', 'B8A': 'B8A'}

band_list = [band_map[ele.text] for ele in bf.find(".//Band_List").findall("BAND_NAME")]
quantification_value = int(bf.find(".//QUANTIFICATION_VALUE").text)
solar_irradiance = [ele.text for ele in bf.find(".//Solar_Irradiance_List").findall("SOLAR_IRRADIANCE")]
physical_gains = [float(ele.text) for ele in bf.findall(".//PHYSICAL_GAINS")]

self.scaling_factor = quantification_value
self.to_radiance = {band: gain for band, gain in zip(band_list, solar_irradiance)}

# get spatial resolution and image sizes
self.spatial_samplings = {}
for size in geo_codings.findall("Size"):
    self.spatial_samplings[float(size.get("resolution"))] = {
        key: int(size.find(key).text) for key in ["NROWS", "NCOLS"]}

# for each type of resolution, get geopositions
for geo in geo_codings.findall("Geoposition"):
    self.spatial_samplings[float(geo.get("resolution"))].update(
        {key: int(geo.find(key).text) for key in ["ULX", "ULY", "XDIM", "YDIM"]})
# build inverse dictionary [shape of image]:spatial sampling
self.shape_to_resolution = {(values["NCOLS"], values["NROWS"]): spatial_sampling for spatial_sampling, values in
                            self.spatial_samplings.items()}

# get file names for each band
self.band_fns = {self._band_name(fn): fn for fn in
                 glob(path.join(S2_MSI_granule_path, "IMG_DATA", "S2A_*.jp2"))}

self.final_shape = [self.spatial_samplings[target_resolution][ii] for ii in ("NROWS", "NCOLS")]
print("Final shape for each channel: %s" % str(self.final_shape))

if data_mode == "dense":
    self.data = __zeros__(shape=self.final_shape + [len(self.band_list)], dtype=numpy_type)
elif data_mode == "sparse":
    self.data = __zeros__(shape=list(self.final_shape) + [len(self.full_band_list)], dtype=numpy_type)
else:
    raise ValueError("data_mode=%s not implemented" % data_mode)

In [ ]:
for iband, band in enumerate(self.band_list):
    t0 = time()
    jpfl = self.__read_img(self.band_fns[band])
    img = np.array(jpfl[:, :], dtype=np.float32) / self.scaling_factor  # / self.to_reflectance[band]
    t1 = time()
    zoom_fac = self.shape_to_resolution[jpfl.shape] / target_resolution

    if data_mode == "dense":
        ii = iband
    elif data_mode == "sparse":
        ii = self.full_band_list.index(band)
    else:
        raise ValueError("data_mode=%s not implemented" % data_mode)

    self.data[:, :, ii] = zoom(input=img, zoom=zoom_fac, order=interpolation_order)